In [1]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import data as dt
import functions as fn
from collections import Counter

In [2]:
BTCUSDT15m = dt.BTCUSDT15m
BTCUSDT15m.set_index("Open Time", inplace=True)

# VWAP SIGNAL

In [3]:
%%time
BTCUSDT15m['VWAP'] = fn.vwap(BTCUSDT15m.High, BTCUSDT15m.Low, BTCUSDT15m.Close, BTCUSDT15m.Volume)

CPU times: user 49.3 ms, sys: 4.04 ms, total: 53.4 ms
Wall time: 56.3 ms


In [4]:
back_candles = 9

In [5]:
BTCUSDT15m['VWAP_buy'] = BTCUSDT15m['Close'] > BTCUSDT15m['VWAP']
BTCUSDT15m['VWAP_sell'] = BTCUSDT15m['Close'] < BTCUSDT15m['VWAP']

BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['VWAP_buy'].rolling(5).sum() >= 5
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['VWAP_buy_signal'].astype(int)
BTCUSDT15m['VWAP_sell_signal'] = BTCUSDT15m['VWAP_sell'].rolling(5).sum() >= 5

In [6]:
BTCUSDT15m['VWAP_sell_signal'] = BTCUSDT15m['VWAP_sell_signal'].map({True: 2 , False: 0})

In [7]:
BTCUSDT15m['VWAP_signal'] = BTCUSDT15m['VWAP_buy_signal']+BTCUSDT15m['VWAP_sell_signal']

# PIVOTS SIGNAL

In [8]:
pivots = fn.pivots(BTCUSDT15m['Open'], BTCUSDT15m['High'], BTCUSDT15m['Low'], BTCUSDT15m['Close'], anchor=15, method='fibonacci')

# SRSI SIGNAL

In [9]:
%%time
BTCUSDT15m = pd.concat([BTCUSDT15m, pta.stochrsi(BTCUSDT15m.Close, 14)], axis = 1)

CPU times: user 47.8 ms, sys: 8.11 ms, total: 55.9 ms
Wall time: 56.9 ms


In [10]:
BTCUSDT15m.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Close Time',
       'Quote Asset Volume', 'Number of Trades', 'TB Base Volume',
       'TB Quote Volume', 'VWAP', 'VWAP_buy', 'VWAP_sell', 'VWAP_buy_signal',
       'VWAP_sell_signal', 'VWAP_signal', 'STOCHRSIk_14_14_3_3',
       'STOCHRSId_14_14_3_3'],
      dtype='object')

In [11]:
BTCUSDT15m.rename(columns = {'STOCHRSIk_14_14_3_3':'K', 'STOCHRSId_14_14_3_3':'D'}, inplace = True)

In [12]:
BTCUSDT15m['SRSI_buy'] = BTCUSDT15m['K'] > BTCUSDT15m['D']
BTCUSDT15m['SRSI_sell'] = BTCUSDT15m['K'] < BTCUSDT15m['D']

BTCUSDT15m['SRSI_buy_signal'] = BTCUSDT15m['SRSI_buy'].rolling(4).sum() >= 4
BTCUSDT15m['SRSI_buy_signal'] = BTCUSDT15m['SRSI_buy_signal'].astype(int)
BTCUSDT15m['SRSI_sell_signal'] = BTCUSDT15m['SRSI_sell'].rolling(4).sum() >= 4

In [13]:
BTCUSDT15m['SRSI_sell_signal'] = BTCUSDT15m['SRSI_sell_signal'].map({True: 2 , False: 0})

In [14]:
BTCUSDT15m['SRSI_signal'] = BTCUSDT15m['SRSI_buy_signal']+BTCUSDT15m['SRSI_sell_signal']

In [15]:
BTCUSDT15m[['VWAP_signal', 'SRSI_signal']]

,VWAP_signal,SRSI_signal
Open Time,,
2018-01-01 00:00:00,0,0
2018-01-01 00:15:00,0,0
2018-01-01 00:30:00,0,0
2018-01-01 00:45:00,0,0
2018-01-01 01:00:00,2,0
...,...,...
2022-07-06 02:45:00,2,0
2022-07-06 03:00:00,2,0
2022-07-06 03:15:00,2,0


# TOTAL SIGNAL

In [16]:
BTCUSDT15m['buy'] = [1 if BTCUSDT15m['VWAP_signal'][i] == 1 and BTCUSDT15m['SRSI_signal'][i] == 1 else 0 for i in range(len(BTCUSDT15m))]

In [18]:
BTCUSDT15m['sell'] = [2 if BTCUSDT15m['VWAP_signal'][i] == 2 and BTCUSDT15m['SRSI_signal'][i] == 2 else 0 for i in range(len(BTCUSDT15m))]

In [20]:
BTCUSDT15m['position'] = BTCUSDT15m['sell']+BTCUSDT15m['buy']

In [30]:
t1 = BTCUSDT15m['2021-03-18 19:00:00': '2021-03-21 00:00:00']
train = BTCUSDT15m['2018-01-01 00:00:00': '2019-01-01 00:00:00']
test = BTCUSDT15m['2019-02-01 00:00:00': '2020-02-01 00:00:00']

In [31]:
%%time
pivs = fn.pivots(t1['Open'], t1['High'], t1['Low'], t1['Close'], anchor=15, method='fibonacci')

CPU times: user 13.8 ms, sys: 8.08 ms, total: 21.9 ms
Wall time: 22.3 ms


In [35]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=t1.index,
            open=t1["Open"],
            high=t1["High"],
            low=t1["Low"],
            close=t1["Close"],
            name="OHLC",
        ),
        go.Scatter(x=t1.index, y=t1['K'], line=dict(color='blue', width=1.5),mode="lines", name="K", yaxis="y2"),
        go.Scatter(x=t1.index, y=t1['D'], line=dict(color='red', width=1.5),mode="lines", name="D", yaxis="y2"),
        go.Scatter(x=t1.index, y=t1.VWAP, line=dict(color='blue', width=1.5), name="VWAP"),
        go.Scatter(x=pivs.index, y=pivs.p, line=dict(color='red', width=1.5), name="P"),
        go.Scatter(x=pivs.index, y=pivs.s1, line=dict(color='orange', width=1), name="S1"),
        go.Scatter(x=pivs.index, y=pivs.s2, line=dict(color='orange', width=1), name="S2"),
        go.Scatter(x=pivs.index, y=pivs.r1, line=dict(color='orange', width=1), name="R1"),
        go.Scatter(x=pivs.index, y=pivs.r2, line=dict(color='orange', width=1), name="R2")]
        ).update_layout(
        yaxis_domain=[0.3, 1],
        yaxis2={"domain": [0, 0.20]},
        xaxis_rangeslider_visible=False,
        showlegend=False,
 )


fig_001

![](files/strategy.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=972d23a0-f342-484b-9096-244a1ebf7fc5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>